# JSON Agent

Agent for interacting with JSON blob

In [1]:
import os
# os.environ["LANGCHAIN_HANDLER"] = "langchain"
import yaml

from langchain.agents import SQLDatabaseAgent, JsonAgent
from langchain.tools.sql_database.toolkit import SQLDatabaseToolkit
from langchain.tools.json.tool import JsonSpec
from langchain.tools.json.toolkit import JsonSpecToolkit
from langchain.tools.requests.toolkit import RequestsToolkit
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.requests import RequestsWrapper
from langchain.agents.tools import Tool
from langchain.agents import Tool, initialize_agent, load_tools
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.chains import LLMChain

In [2]:
with open("../../../../notebooks/openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
}
request_toolkit = RequestsToolkit(requests_wrapper=RequestsWrapper(headers=headers))
json_toolkit = JsonSpecToolkit(spec=JsonSpec(dict_=data, max_value_length=4000))

json_agent = JsonAgent.from_llm_and_tools(
    llm=OpenAI(temperature=0),
    tools=json_toolkit.get_tools(),
)
json_agent_executor = AgentExecutor.from_agent_and_tools(
    agent=json_agent, tools=json_toolkit.get_tools(), verbose=True
)

In [4]:
json_agent_executor.run("What are the required parameters in the request body to the /completions endpoint?")



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the paths key to see what endpoints exist
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/engines', '/engines/{engine_id}', '/completions', '/edits', '/images/generations', '/images/edits', '/images/variations', '/embeddings', '/engines/{engine_id}/search', '/files', '/files/{file_id}', '/files/{file_id}/content', '/answers', '/classifications', '/fine-tunes', '/fine-tunes/{fine_tune_id}', '/fine-tunes/{fine_tune_id}/cancel', '/fine-tunes/{fine_tune_id}/events', '/models', '/models/{model}', '/moderations']
Thought: I should look at the /completions endpoint to see what parameters are required
Action: json_spec_list_keys
Action Input: data["paths"]["/completions"]
Observation: ['post']
Thought: I should look at the post key to see what parameters are required
Actio

"The required parameters in the request body to the /completions endpoint are 'model'."

In [5]:
json_agent_executor.run("Write me an example request to the completions endpoint using curl. The prompt should be 'tell me a joke'.")



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the paths key to see what endpoints I have access to
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/engines', '/engines/{engine_id}', '/completions', '/edits', '/images/generations', '/images/edits', '/images/variations', '/embeddings', '/engines/{engine_id}/search', '/files', '/files/{file_id}', '/files/{file_id}/content', '/answers', '/classifications', '/fine-tunes', '/fine-tunes/{fine_tune_id}', '/fine-tunes/{fine_tune_id}/cancel', '/fine-tunes/{fine_tune_id}/events', '/models', '/models/{model}', '/moderations']
Thought: I should look at the completions endpoint
Action: json_spec_get_value
Action Input: data["paths"]["/completions"]
Observation: {'post': {'operationId': 'createCompletion', 'tags': ['OpenAI'], 'summary': 'Creates a completion for the provided 

'curl https://api.openai.com/v1/completions -H \'Content-Type: application/json\' -H \'Authorization: Bearer YOUR_API_KEY\' -d \'{"model": "VAR_model_id", "prompt": "Tell me a joke", "max_tokens": 7, "temperature": 0}\''

## Interfacing json agent with requests

In [3]:
llm = OpenAI(temperature=0)
tools = [
    *request_toolkit.get_tools(),
    Tool(
        name="json_explorer",
        func=json_agent_executor.run,
        description="""
        Can be used to answer questions about the openapi spec for the API. Always use this tool before trying to make a request. 
        Example input to this tool: 'What are the required parameters in the request body to the /foo endpoint?'
        Always give this tool a specific question.
        """
    )
]

In [36]:
prefix = """
You are an agent designed to make web requests to an API given the openapi spec.

If the question does not seem related to the API, return I don't know. Do not make up an answer.
Only use information provided by the tools to construct your response.

Ensure that you are sending the correct parameters to the request by checking which parameters are required.
For parameters with a fixed set of values, please use the spec to look at which values are allowed.
"""
suffix = """Begin!"

Question: {input}
Thought: I should look for the base url I need to make my requests.
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [37]:
agent_executor.run("Make a request to openai /completions. The prompt should be 'tell me a joke.'")



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the /completions endpoint?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the 'servers' key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, use `value` instead to get its value directly.')
Thought: I should use the json_spec_get_value tool to get the value of the 'servers' key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.openai.com/v1'}
Thought: I now know the base url for the /completions endpoint
Final Answer: The base url for the /completions endpoint is https://api.openai.com/v1

> Finished chain.

Observation: The base url for the /completions endpoint is https://api.o

'The response to the request to openai /completions with the prompt \'tell me a joke\' is {"id":"cmpl-6mzgH11ulT5jSqds74DQZgu9KU6OF","object":"text_completion","created":1677134865,"model":"davinci","choices":[{"text":"”—the very pithy antithesis of my perceptions of life","index":0,"logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":4,"completion_tokens":15,"total_tokens":19}}.'

In [26]:
with open("../../../../notebooks/openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
}
request_toolkit = RequestsToolkit(requests_wrapper=RequestsWrapper(headers=headers))
json_toolkit = JsonSpecToolkit(spec=JsonSpec(dict_=data, max_value_length=4000))

json_agent = JsonAgent.from_llm_and_tools(
    llm=OpenAI(temperature=0),
    tools=json_toolkit.get_tools(),
)
json_agent_executor = AgentExecutor.from_agent_and_tools(
    agent=json_agent, tools=json_toolkit.get_tools(), verbose=True
)

llm = OpenAI(temperature=0)
tools = [
    *request_toolkit.get_tools(),
    Tool(
        name="json_explorer",
        func=json_agent_executor.run,
        description="""
        Can be used to answer questions about the openapi spec for the API. Always use this tool before trying to make a request. 
        Example inputs to this tool: 
            'What are the required parameters in the request body for a POST request to the /foo endpoint?'
            'What are the required query parameters for a GET request to the /bar endpoint?`
        Always give this tool a specific question.
        """
    )
]

prefix = """
You are an agent designed to answer questions by making web requests to an API given the openapi spec.

If the question does not seem related to the API, return I don't know. Do not make up an answer.
Only use information provided by the tools to construct your response.

First, find the base URL needed to make the request.

Second, find the relevant paths needed to answer the question. Take note that, sometimes, you might need to make more than one request to more than one path to answer the question.

Third, find the required parameters needed to make the request. For GET requests, these are usually URL parameters and for POST requests, these are request body parameters.

Fourth, make the requests needed to answer the question. Ensure that you are sending the correct parameters to the request by checking which parameters are required. For parameters with a fixed set of values, please use the spec to look at which values are allowed.

Use the exact parameter names as listed in the spec, do not make up any names or abbreviate the names of parameters.
If you get a not found error, ensure that you are using a path that actually exists in the spec.
"""
suffix = """Begin!"

Question: {input}
Thought: I should explore the spec to find the base url for the API.
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [27]:
# agent_executor.run("Make a request to get current temperature at latitude 52.52 longitude 13.41 in farenheit?")
agent_executor.run("Post to /completions. The prompt should be 'tell me a joke.'")



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the servers key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, use `json_spec_get_value` instead to get its value directly.')
Thought: I should use json_spec_get_value to get the value of the servers key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.openai.com/v1'}
Thought: I now know the final answer
Final Answer: The base url for the API is https://api.openai.com/v1

> Finished chain.

Observation: The base url for the API is https://api.openai.com/v1
Thought: I should find the path and parameters needed to make the 

'"\\", e.g. hika digu skis, but this is nice'